In [1]:
%matplotlib qt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import os
from scipy.stats import binned_statistic
from scipy.optimize import curve_fit
import concurrent
import pickle
from scipy.interpolate import griddata
from scipy import interpolate

import matplotlib
from mpl_toolkits import mplot3d
font = {'size'   : 16}
matplotlib.rc('font', **font)

In [5]:
#file_path = r"C:\Users\lukas\OneDrive - University of Cambridge\PhD\3DMOKE\Data\MagnetCalibration\hex_minor_Ichannel.h5"
file_path = r"C:\Users\3DStation4\PycharmProjects\pythonProject_3DMOKE_new\hex_minor_Ichannel.h5"
print(os.path.isfile(file_path))
out_folder = 'results_forward'
try:
    os.stat(out_folder)
except:
    os.mkdir(out_folder) 
poles = [0, 1, 2]
poles_dict = {0:'A', 1:'B', 2:'C'}

True


In [6]:
# poles = [0]
hyst_all = []
hyst_file_path = file_path
complianceV = 9 # if the current is too high we hit compliance in the voltage (i.e. trying to apply a current which requires a too high voltage).
                #This is approx 20V/2.1Ohm
with h5py.File(hyst_file_path, 'r') as f:
    for pole in poles:
        hyst_pole = []
        pole_grp = f['pole{}'.format(pole)]
        for step in pole_grp.keys():
            hp = pd.DataFrame(pole_grp[step+'/hallprobe/data'][:])
            hx = pd.DataFrame(pole_grp[step+'/hexapole/data'][:])
            bins = np.arange(hp[0].iloc[0], hp[0].iloc[-1], 1/100)

            hp['bins'] = pd.cut(hp[0], bins, right=False)
            hp = hp.groupby('bins').mean().reset_index().drop(labels=[0,'bins'], axis=1)
            hp.columns = ['hpA', 'hpB', 'hpC']
            hx['bins'] = pd.cut(hx[0], bins, right=False)
            hx = hx.groupby('bins').mean().reset_index().drop(labels=[0,'bins'], axis=1)
            hx.columns = ['hxA', 'hxB', 'hxC']

            # add to one dataframe, flip hp for convenience
            hyst_step = pd.concat([-hp, hx], axis=1)
            # remove the part where we hit compliance
#             hyst_step[hx.columns] = hyst_step[hyst_step[hx.columns].abs()<9][hx.columns]
#             hyst_step.dropna(inplace=True)
            
            # add to the list
            hyst_pole.append(hyst_step.copy())
        # sort the list based on the amplitude
        hx_amps = np.zeros(len(hyst_pole))
        for i, df in enumerate(hyst_pole):
            hx = df['hx'+poles_dict[pole]]
            hx_amps[i] = hx.max() - hx.min()
        # sort the hyst pole based on amps
        hyst_pole = [x for _,x in sorted(zip(hx_amps,hyst_pole))]

        # add to the pole list
        hyst_all.append(hyst_pole)

C:\Users\3DStation4\AppData\Local\Temp\ipykernel_17868\3878252941.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  hp = hp.groupby('bins').mean().reset_index().drop(labels=[0,'bins'], axis=1)
C:\Users\3DStation4\AppData\Local\Temp\ipykernel_17868\3878252941.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  hx = hx.groupby('bins').mean().reset_index().drop(labels=[0,'bins'], axis=1)
C:\Users\3DStation4\AppData\Local\Temp\ipykernel_17868\3878252941.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain cur

In [7]:
# plot everything
plt.close('all')
plt.figure(figsize=(16, 8))
for pole, hyst_pole in enumerate(hyst_all):
    plt.subplot(1, len(poles), pole+1)
    for i, hyst_step in enumerate(hyst_pole):
        plt.plot(hyst_step['hx'+poles_dict[pole]],hyst_step['hp'+poles_dict[pole]], label='step{}'.format(i))
        plt.ylabel('Measured field [V]')
        plt.xlabel('Applied current [A]')
#     plt.legend()
    plt.title('Pole {}'.format(poles_dict[pole]))

plt.tight_layout()
plt.savefig(os.path.join(out_folder, 'Magnet minor loops.png'))

In [8]:
v2mt = 27.26
# plot everything
plt.close('all')
plt.figure(figsize=(16, 8))
for pole, hyst_pole in enumerate(hyst_all):
    plt.subplot(1, len(poles), pole+1)
    for i, hyst_step in enumerate(hyst_pole):
        plt.plot(hyst_step['hx'+poles_dict[pole]], v2mt*hyst_step['hp'+poles_dict[pole]], label='step{}'.format(i))
        plt.ylabel('Measured field [V]')
        plt.xlabel('Applied current [A]')
#     plt.legend()
    plt.title('Pole {}'.format(poles_dict[pole]))
    plt.ylim([-60, 60])
    plt.yticks(np.arange(-50, 55, 10))

plt.tight_layout()
plt.savefig(os.path.join(out_folder, 'Magnet minor loops calibrated.png'))

### Fitting the bounding loop (f<sub>a</sub>)

In [9]:
plt.close('all')
plt.figure(figsize=(16, 8))
main_branch_fits = []

for pole in poles:
    bounding_loop = hyst_all[pole][-1]
    x_full = bounding_loop['hx'+poles_dict[pole]].values
    y_full = bounding_loop['hp'+poles_dict[pole]].values
    # take only the top half
    x = x_full[int(np.floor(len(x_full)/2)):]
    y = y_full[int(np.floor(len(y_full)/2)):]

    # fit the hysteresis
    idx = np.argsort(x)
    x, y = x[idx], y[idx]
    spl = interpolate.UnivariateSpline(x, y, s=0.01)
#     spl = interpolate.InterpolatedUnivariateSpline(x, y)
    main_branch_fits.append(spl)
    y_fit = spl.__call__(x)
    err = np.abs((y_fit - y))
    idx = np.argmax(err)
    print('Max error: {}'.format(err[idx]))
#     print(y[idx])
    
#     coeff = np.polyfit(x, y, deg=12)
#     f_a = np.poly1d(coeff)
    
    plt.subplot(1, len(poles), pole+1)
    plt.plot(x_full, y_full, label='Measurement')
    x_tst = np.linspace(-10, 10, 1000)
    plt.plot(x_tst, spl.__call__(x_tst), label='Fit')
    plt.legend()
    plt.xlabel('Applied current [A]')
    plt.ylabel('Measured field [V]')
    
#     plt.plot(x, f_a(x), label='Fit')
    
plt.tight_layout()
plt.savefig(os.path.join(out_folder, 'Fitting bounding loop all poles.png'))

Max error: 0.03104196890597577
Max error: 0.044382481858063816
Max error: 0.03333739632360877


In [10]:
f = 1
Z = np.sqrt(2**2+(2*np.pi*f*0.2)**2)
print(20/Z)

8.467330159648304


In [11]:
with open('tst.p', 'wb') as f:
    pickle.dump(spl, f)

### Fitting the minor loops f<sub>ab

In [12]:
# for each of the transition curves, get the value maximum value it gets to
transition_curves = []
for pole in poles:
    # maximum value of the minor loops
    minor_loops = hyst_all[pole]
    # a = np.array([minor_loop['hx'+poles_dict[pole]].max() for minor_loop in minor_loops])
    # get the list of returning loops
    tran_curves = []
    for ml in minor_loops:
        y = ml['hp'+poles_dict[pole]]
        x = ml['hx'+poles_dict[pole]]
        # get the maximum value of the loop
        a = x.max()
        # take only the bottom half
        b = x[int(np.floor(len(x)/2)):].values
        f_ab = y[int(np.floor(len(y)/2)):].values
        tran_curves.append(np.stack((a*np.ones(b.size), b, f_ab), axis=1))
    transition_curves.append(np.concatenate(tran_curves, axis=0))

In [13]:
plt.close('all')
fig = plt.figure(figsize=(16, 8))
transition_fits = []
poles = [0,1,2]
for pole in poles:
    tran_pole = transition_curves[pole]
    x, y, z = tran_pole[:, 0], tran_pole[:, 1], tran_pole[:, 2]
    grid_x, grid_y = np.mgrid[-10:10:200j, -10:10:200j]
#     fit = interpolate.interp2d(x, y, z, kind='cubic')
    fit = interpolate.SmoothBivariateSpline(x, y, z, s=2, kx=3, ky=3)
    transition_fits.append(fit)
    grid_z = fit.__call__(grid_x[:, 0], grid_y[0, :])
#     grid_z = np.reshape(fit(grid_x.flatten(), grid_y.flatten()), grid_x.shape)

    # for illustration purposes, make the grid z be 0 outside of the relevant area
    idx = grid_x<grid_y
    grid_z[idx] = 0
    
    grid_z = np.reshape(grid_z, grid_x.shape)

    ax = fig.add_subplot(1, len(poles), 1+pole, projection='3d')
    ax.scatter(x, y, z)
    ax.plot_surface(grid_x, grid_y, grid_z, alpha=0.5, color='red')
    ax.set_zlim([-2, 2])
    ax.set_xlabel('a')
    ax.set_ylabel('b')
    ax.set_zlabel('f_ab')

    ax.view_init(15, -140)
    plt.tight_layout()
    z_eval = fit.ev(x, y)
#     z_eval = fit(x, y)
    print('Maximum error: ', np.max(np.abs((z-z_eval))))
#     break
plt.savefig(os.path.join(out_folder, 'Fitting transition curves all poles.png'))

Maximum error:  0.16269720602401028
Maximum error:  0.16463094571146042
Maximum error:  0.1577970297144039


### Define the triangle function F(a, b)

In [14]:
# define the triangle functions:
F_functions = [lambda a, b, f_a=f_a, f_ab=f_ab: (f_a.__call__(a) - f_ab.ev(a, b))/2 for f_a, f_ab in zip(main_branch_fits, transition_fits)]
# check that it gives sensible results
print(F_functions[0](1.8, 1.8))
print(main_branch_fits[1].__call__(1.8))
print(transition_fits[1].ev(1.8, -1.8))

-0.12282446663382202
-1.0350434739330288
0.5308298976829822


### Create functions for updating the line points of the Prezbiach model

In [15]:
from numba import jit
from itertools import groupby
# define the starting line
# L = np.array([[10, -10], [-10, -10]])

@jit(nopython=True)
def update_line(L, x):
    # first find out if we went up or down
    if L[-1, 0] < x:
        # we went up, get rid of the history of all elements that have alpha smaller than x
        L[L[:, 0] < x, 0] = x
        # add the newest element
        L = np.vstack((L, np.array([[x, x]])))
        # every other element needs to have a different alpha, otherwise need to get rid of it
        keep = np.zeros(L.shape[0]).astype(np.bool_)
        keep[0] = True
        keep[-1] = True
        for i in range(L.shape[0]-2):
            if L[i, 0] != L[i+1, 0]:
                keep[i] = True
                keep[i+1] = True
        L = L[keep, :]
        return L
    if L[-1, 0] > x:
        # we went up, get rid of the history of all elements that have alpha smaller than x
        L[L[:, 1] > x, 1] = x
        # add the newest element
        L = np.vstack((L, np.array([[x, x]])))
        # every other element needs to have a different alpha, otherwise need to get rid of it
        keep = np.zeros(L.shape[0]).astype(np.bool_)
        keep[0] = True
        keep[-1] = True
        for i in range(L.shape[0]-2):
            if L[i, 1] != L[i+1, 1]:
                keep[i] = True
                keep[i+1] = True
        L = L[keep, :]
        return L
    return L    

def get_signal_value(L, F_fun):
    """Gets the value of the signal having the points of the Prezbiach line and the integral functions"""
    # get alphas and betas
    alpha, beta = L[:, 0], L[:, 1]
    # get all the horisontal lines
    indx_hor = (beta[1:] - beta[:-1]) != 0
#     lenh = np.sum(indx_hor)
#     print(lenh)
    # calcluate the signal
    signal = -F_fun(alpha[0], beta[0])
    F_kk = F_fun(alpha[:-1][indx_hor], beta[:-1][indx_hor])
    F_kk1 = F_fun(alpha[1:][indx_hor], beta[1:][indx_hor])
    signal += 2*np.sum(F_kk - F_kk1)    
    return signal

def reconstruct_signal(signal, L_start, pole):
    # define the starting line
    L = L_start
    signal_out = np.zeros(len(signal))
    for i, x in enumerate(signal):
        L = update_line(L, x)
        signal_out[i] = get_signal_value(L, F_functions[pole])
    return signal_out

### Try to reconstruct the signal

In [18]:
L = np.array([[10, -10], [-10, -10]]).astype(np.float64)
L = update_line(L, 5)
L = update_line(L, 3)
print(L)
# L = update_line(L, 9.9)
# print(L)
# L = update_line(L, 9)
# print(L)
# L = update_line(L, -5)
# print(L)

[[ 10. -10.]
 [  5. -10.]
 [  5.   3.]
 [  3.   3.]]


In [19]:
get_signal_value(L, F_functions[0])

-0.9096918225492634

In [21]:

L_start = np.array([[10, -10], [-10, -10]]).astype(np.float64)
pole = 0
y = hyst_all[pole][-1]['hp'+poles_dict[pole]].values
x = hyst_all[pole][-1]['hx'+poles_dict[pole]].values
y_pred = reconstruct_signal(x, L_start, pole)

In [22]:
plt.close('all')
idx = np.ones(x.size).astype(bool)
# idx[:int(len(idx)/2)] = True
plt.plot(x[idx], y[idx], label='Data')
plt.plot(x[idx], y_pred[idx], label='Fit')
plt.plot(x, y-y_pred, label='Error')
# idx = np.logical_not(idx)
# plt.plot(x[idx], y[idx], label='Data')
# plt.plot(x[idx], y_pred[idx], label='Fit')
plt.xlabel('Applied signal [V]')
plt.ylabel('Measured field [V]')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(out_folder, 'Fitting hysteresis.png'))

## Save all the results

In [23]:
hysteresis_fits = {'main_branch':main_branch_fits, 'transition_branches':transition_fits}
with open('hysteresis_fits.p', 'wb') as f:
    pickle.dump(hysteresis_fits, f)